In [204]:
import pandas as pd
import datetime
from tabulate import tabulate

In [139]:
df1 = pd.read_csv('camp_database.csv')
df1.dropna(how="all", inplace=True)
df1.fillna('',inplace=True)

df2 = pd.read_csv('emergency_database.csv')
df2.dropna(how="all", inplace=True)
df2.fillna('',inplace=True)

df3 = pd.read_csv("countries.csv", index_col = 'Country name')

df4 = pd.read_csv('volunteer_database.csv')
df4.dropna(how="all", inplace=True)
df4.fillna('',inplace=True)

df5 = pd.read_csv('refugee_database.csv')
df5.dropna(how="all", inplace=True)
df5.fillna('',inplace=True)

df6 = pd.read_csv('user_database.csv')
df6.dropna(how="all", inplace=True)
df6.fillna('',inplace=True)


In [140]:
class test():
    
    def __init__(self):
        self.user_db = None
        self.vol_db = None
        self.refugee_db = None
        self.camps_db = None
        self.countries_db = None
        self.emergencies_db = None 

        fileCheckError =  self.download_all_data()
        
        if fileCheckError:
            exit()

        self.current_user = 'Volunteer1'
        self.camp_of_user = 'AU1'
        self.functions = 10
        pass
    
    def download_all_data(self):
        
        dataFailure = False

        try: 
            df = pd.read_csv('user_database.csv')
            df.dropna(how="all", inplace=True)
            df.fillna('',inplace=True)
            df['password'] = df['password'].astype(str)
            self.user_db = df
        except FileNotFoundError:
            user_db = {'username':['admin'],'password':['111'],'role':['admin'],'activated':['TRUE']}
            df = pd.DataFrame(user_db)
            df.set_index('username', inplace=True)
            df['password'] = df['password'].astype(str)
            df.to_csv('user_database.csv')
            self.user_db = df
        except:
            print("System couldn't read your user database file.")
            dataFailure = True 

        try:
            df = pd.read_csv('volunteer_database.csv')
            df.dropna(how="all", inplace=True)
            df.fillna('',inplace=True)
            self.vol_db = df
        except FileNotFoundError:
            vol_db = {'Username':[''],'First name':[''],'Second name':[''],'Camp ID':[''],'Avability':[''],'Status':['']}
            df = pd.DataFrame(vol_db)
            df.set_index('Username', inplace=True)
            df.to_csv('volunteer_database.csv')
            self.vol_db = df
        except:
            print("System couldn't read your volunteer database file.")
            dataFailure = True

        try:
            df = pd.read_csv('refugee_database.csv')
            df.dropna(how="all", inplace=True)
            df.fillna('',inplace=True)
            self.refugee_db = df
        except FileNotFoundError:
            refugee_db = {'Family ID':[''],'Lead Family Member Name':[''],'Lead Family Member Surname':[''],'Camp ID':[''],'Mental State':[''],'Physical State':[''],'No. Of Family Members':['']}
            df = pd.DataFrame(refugee_db)
            df.set_index('Family ID', inplace=True)
            df.to_csv('refugee_database.csv')
            self.refugee_db = df
        except:
            print("System couldn't read your refugees database file.")
            dataFailure = True

        try:
            df = pd.read_csv('camp_database.csv')
            df.dropna(how="all", inplace=True)
            df.fillna('',inplace=True)
            self.camps_db = df
        except FileNotFoundError:
            camps_db = {'Emergency ID':[''],'Type of emergency':[''],'Description':[''],'Location':[''],'Start date':[''],'Close date':[''],'Number of refugees':[''],'Camp ID':[''],'No Of Volounteers':[''],'Capacity':['']}
            df = pd.DataFrame(camps_db)
            df.set_index('Emergency ID', inplace=True)
            df.to_csv('camp_database.csv')
            self.camps_db = df
        except:
            print("System couldn't read your camplist database file.")
            dataFailure = True

        try:
            df = pd.read_csv('emergency_database.csv')
            df.dropna(how="all", inplace=True)
            df.fillna('',inplace=True)
            self.emergencies_db = df
        except FileNotFoundError:
            emergencies_db = {'Emergency ID':[''],'Location':[''],'Type':[''],'Description':[''],'Start date':[''],'Close date':['']}
            df = pd.DataFrame(emergencies_db)
            df.set_index('Emergency ID', inplace=True)
            df.to_csv('emergency_database.csv')
            self.emergencies_db = df
        except:
            print("System couldn't read your camplist database file.")
            dataFailure = True
        
        try:
            df = pd.read_csv("countries.csv", index_col = 'Country name')
            self.countries_db = df
        except:
            print("System couldn't read the countries database file.")
            dataFailure = True
        
        return dataFailure
    
    def count_ref_vol(self):  # what is the purpose of this method?
        '''
        Counts the number of volunteers in each camp and the number of refugees in each camp, after which it updates the camp_database.csv with correct numbers.
        '''
        camp_df = self.camps_db.copy()
        refugee_df = self.refugee_db.copy()
        vol_df = self.vol_db.copy()

        count_vol = vol_df['Camp ID'].value_counts()
        count_ref = refugee_df['Camp ID'].value_counts()

        for i in count_vol.index:
            camp_df.loc[camp_df['Camp ID'] == i,
                        'Number of volunteers'] = count_vol[i]
        for i in count_ref.index:
            camp_df.loc[camp_df['Camp ID'] == i,
                        'Number of refugees'] = count_ref[i]

        self.camps_db = camp_df.copy()
        camp_df.to_csv('camp_database.csv', index=False)

    def write_volunteer(self):
        '''
        Allows admin to create one or more empty volunteer accounts.

        Either manual input can be selected. 
            INPUTS: username, password and camp of volunteer
        Or automatic creation of multiple volunteer accounts from the same camp can be made which have automatically generated usernames in form "Volunteer"+index
            INPUTS: number of volunteers desired, camp of volunteers
        '''
        vol_df = self.vol_db.copy()
        users_df = self.user_db.copy()
        camps_df = self.camps_db.copy()
        users_exist = list(users_df['username'])
        camps_exist = list(camps_df['Camp ID'])
        self.quit = False
        print(100*'=')
        print('Please select how would you like to create a new volunteer profile')
        print('[1] - manual input')
        print('[2] - automatic creation')
        print('[B] to go back')
        print('[Q] to quit')

        def manual():
            counter = 0
            while True:

                def assign_username():
                    global username
                    while True:
                        inpt = input('\nEnter a new username: ')
                        if inpt in users_exist:
                            print('Username taken. Try another.')
                            continue
                        break
                    username = inpt
                    if inpt == 'Q' or inpt == 'B':
                        print(100*'=')
                        menu(self.functions)
                        exit()
                    return 1

                def assign_password():
                    global password
                    inpt = input('\nSet a password: ')
                    password = inpt
                    if inpt == 'B':
                        return -1
                    elif inpt == 'Q':
                        print(100*'=')
                        menu(self.functions)
                        exit()
                    else:
                        return 1

                def assign_camp():
                    global camp
                    while True:
                        print('\nSet camp:')
                        print('[1] - View camp summary information')
                        print('[2] - Assign camp')
                        print('[B] to go back')
                        print('[Q] to quit')
                        user_input = input('\nChoose interaction: ')
                        if user_input == '1':
                            print(camps_df)
                        elif user_input == '2':
                            pass
                        elif user_input == 'B' or user_input == 'Q':
                            break
                        else:
                            print('Invalid input.')
                            continue
                        user_input = input('\nEnter camp ID: ')
                        while user_input not in camps_exist:
                            print('Invalid Camp ID.')
                            user_input = input('\nEnter camp ID: ')
                        camp = user_input
                        break

                    if user_input == 'B':
                        return -1
                    elif user_input == 'Q':
                        print(100*'=')
                        menu(self.functions)
                        exit()
                    else:
                        return 1

                inputs = [assign_username, assign_password, assign_camp]

                while counter < len(inputs):
                    counter += inputs[counter]()

                vol_df.loc[len(vol_df.index)] = [
                    username, '', '', '', camp, '']
                users_df.loc[len(users_df.index)] = [
                    username, password, 'volunteer', 'TRUE', '']
                print(users_df.tail(1))
                while True:
                    commit = input('\nCommit changes? [y]/[n] ')
                    if commit == 'y' or commit == 'n':
                        break
                    else:
                        print('Your input is not recognised')
                        continue

                if commit == 'y':
                    self.vol_db = vol_df.copy()
                    self.user_db = users_df.copy()
                    vol_df.to_csv('volunteer_database.csv', index=False)
                    users_df.to_csv('user_database.csv', index=False)
                else:
                    counter = 0
                    continue

                while True:
                    repeat = input(
                        '\nWould you like to create another volunteer? [y]/[n] ')
                    if repeat == 'y' or repeat == 'n':
                        break
                    else:
                        print('Your input is not recognised')
                        continue
                if repeat == 'n':
                    break
                else:
                    counter = 0
                    continue

        def automatic():
            while True:
                while True:
                    no_of_new_users = input(
                        '\nPlease select the number of new volunteers you wish to create: ')
                    try:
                        no_of_new_users = int(no_of_new_users)
                        if no_of_new_users == 'Q':
                            print(100*'=')
                            menu(self.functions)
                            exit()
                    except ValueError:
                        print('Your input needs to be an integer')
                        continue
                    while True:
                        camp = input('\nEnter camp ID: ')
                        if camp not in camps_exist:
                            print('Invalid Camp ID.')
                            continue
                        else:
                            break
                    if camp == 'B':
                        continue
                    elif no_of_new_users == 'Q':
                        menu(self.functions)
                        exit()
                    else:
                        break

                new_usr_index = len(vol_df.index)+1
                for i in range(no_of_new_users):
                    vol_df.loc[len(vol_df.index)] = [
                        'Volunteer'+str(new_usr_index+i), '', '', '', camp, '']
                    users_df.loc[len(users_df.index)] = [
                        'Volunteer'+str(new_usr_index+i), '111', 'volunteer', 'TRUE', '']

                print(users_df.tail(no_of_new_users))
                while True:
                    commit = input('\nCommit changes? [y]/[n] ')
                    if commit == 'y' or commit == 'n':
                        break
                    else:
                        print('Your input is not recognised')
                        continue

                if commit == 'y':
                    self.vol_db = vol_df.copy()
                    self.user_db = users_df.copy()
                    vol_df.to_csv('volunteer_database.csv', index=False)
                    users_df.to_csv('user_database.csv', index=False)
                    break
                else:
                    continue

        while True:
            user_input = input('Choose interaction: ')
            if user_input == '1' or user_input == '2':
                break
            else:
                print('You input is not recognised')
                continue

        if user_input == '1':
            manual()
        else:
            automatic()
        print(100*'=')

def menu(a):
    print('You in menu')
    
tst = test()
tst.write_volunteer()
tst.count_ref_vol()
print(tst.vol_db)
print(tst.camps_db)

Please select how would you like to create a new volunteer profile
[1] - manual input
[2] - automatic creation
[B] to go back
[Q] to quit
Choose interaction: Q
You input is not recognised
Choose interaction: B
You input is not recognised
Choose interaction: 1

Enter a new username: Vol

Set a password: 111

Set camp:
[1] - View camp summary information
[2] - Assign camp
[B] to go back
[Q] to quit

Choose interaction: 2

Enter camp ID: AU1-1
   username password       role activated email
44      Vol      111  volunteer      TRUE      

Commit changes? [y]/[n] y

Would you like to create another volunteer? [y]/[n] n
       Username First name Second name      Phone Camp ID Availability
0    Volunteer1      Fedor       Green  441234567   AU1-1          15h
1    Volunteer2       Jane      Smith   449876543   AU1-1          40h
2    Volunteer3     Meghan      Brown   441312456   AU1-2          10h
3    Volunteer4       Sam     Richards  446745891   AU1-1          30h
4    Volunteer5      A

In [143]:
str(max([int(i[0]) for i in list(tst.refugee_db['Family ID']) if 'AU1-1' in i])+1)

'4'

In [152]:
index = []
for i in list(tst.refugee_db.loc[tst.refugee_db['Camp ID']=='AU1-1']['Family ID']):
    for chr in i:
        if chr.isalpha():
            temp = i.index(chr)
            index.append(i[:temp])
            break

In [173]:
tst.refugee_db

,Family ID,Lead Family Member Name,Lead Family Member Surname,Camp ID,Mental State,Physical State,No. Of Family Members
0,duck,JOHN,Collins,AU1-1,good,bad,3
1,1AU1-2,Khan,Samuel,AU1-2,good,bad,4
2,2AU1-1,Mark,Brown,AU1-1,bad,bad,2
3,3AU1-1,Michael,Smith,AU1-1,good,good,3
4,2AU1-2,Sam,Roberts,AU1-2,bad,good,5
5,3AU1-2,Sophie,Jones,AU1-2,bad,bad,2


In [171]:
tst.refugee_db[tst.refugee_db["Camp ID"] == 'AU1-1']

,Family ID,Lead Family Member Name,Lead Family Member Surname,Camp ID,Mental State,Physical State,No. Of Family Members
0,duck,JOHN,Collins,AU1-1,good,bad,3
2,2AU1-1,Mark,Brown,AU1-1,bad,bad,2
3,3AU1-1,Michael,Smith,AU1-1,good,good,3


In [182]:
tst.camps_db[tst.camps_db["Emergency ID"] == 'AU1']['Camp ID'].values[0]

'AU1-1'

In [183]:
choose_emergency = 'AU1'
camp_id = tst.camps_db[tst.camps_db["Emergency ID"] == choose_emergency]['Camp ID'].values[0]
group_camps = tst.refugee_db[tst.refugee_db["Camp ID"] == camp_id].groupby("Camp ID")
for name, camp in group_camps:
    print("\nCamp " + name + " --> " + str(len(camp)) + " family/families")
    print('\n',camp)


Camp AU1-1 --> 3 family/families

   Family ID Lead Family Member Name Lead Family Member Surname Camp ID  \
0      duck                    JOHN                    Collins   AU1-1   
2    2AU1-1                    Mark                      Brown   AU1-1   
3    3AU1-1                 Michael                      Smith   AU1-1   

  Mental State Physical State  No. Of Family Members  
0         good            bad                      3  
2          bad            bad                      2  
3         good           good                      3  


In [186]:
camps = list(tst.camps_db['Camp ID'])

['AU1-1', 'AU1-2', 'PK1-1', 'PK1-2', 'AU1-3']

In [190]:
camps = list(tst.camps_db.loc[tst.camps_db['Number of refugees'] != '','Camp ID'])

In [194]:
tst.camps_db.loc[tst.camps_db['Camp ID'] != 'AU1-1','Number of refugees'].values[0]

3

In [ ]:
print(tabulate(df, headers='keys', tablefmt='psql'))

In [209]:
camps = list(tst.camps_db.loc[tst.camps_db['Number of refugees'] != '','Camp ID'])
for i in camps:
    no_of_families = str(tst.camps_db.loc[tst.camps_db['Camp ID'] != 'AU1-1','Number of refugees'].values[0])
    refugees_in_camp = tst.refugee_db.loc[tst.refugee_db['Camp ID'] == i]
    
    print("\n\tCamp " + i + " --> " + no_of_families + " family/families")
    print('\n',tabulate(refugees_in_camp, headers='keys', tablefmt='psql', showindex=False))
    print(100*'-')


	Camp AU1-1 --> 3 family/families

 +-------------+---------------------------+------------------------------+-----------+----------------+------------------+-------------------------+
| Family ID   | Lead Family Member Name   | Lead Family Member Surname   | Camp ID   | Mental State   | Physical State   |   No. Of Family Members |
|-------------+---------------------------+------------------------------+-----------+----------------+------------------+-------------------------|
| duck        | JOHN                      | Collins                      | AU1-1     | good           | bad              |                       3 |
| 2AU1-1      | Mark                      | Brown                        | AU1-1     | bad            | bad              |                       2 |
| 3AU1-1      | Michael                   | Smith                        | AU1-1     | good           | good             |                       3 |
+-------------+---------------------------+--------------------------

In [210]:
df = ['a','b','c']
print('\n',tabulate(df, headers='keys', tablefmt='psql', showindex=False))


 +-----+
| 0   |
|-----|
| a   |
| b   |
| c   |
+-----+


In [223]:
df = tst.camps_db.loc[tst.camps_db['Number of volunteers'] != '', ['Camp ID','Number of volunteers']]
print('\n',tabulate(df, headers='keys', tablefmt='psql', showindex=False))


 +-----------+------------------------+
| Camp ID   |   Number of volunteers |
|-----------+------------------------|
| AU1-1     |                     30 |
| AU1-2     |                     13 |
| PK1-1     |                      1 |
| PK1-2     |                     50 |
| AU1-3     |                     10 |
+-----------+------------------------+


In [246]:
df = tst.camps_db.Location.value_counts().reset_index()
df.rename(columns = {'index':'Location', 'Location':'No of camps'}, inplace = True)
print("\nCamps in each area: \n",tabulate(df, headers='keys', tablefmt='psql', showindex=False))


Camps in each area: 
 +------------+---------------+
| Location   |   No of camps |
|------------+---------------|
| Australia  |             3 |
| Pakistan   |             2 |
+------------+---------------+


In [245]:
df = self.emergencies_db.groupby(by=self.emergencies_db['Type']).size()
#print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

Location
Australia    3
Pakistan     2
dtype: int64

In [247]:
tst.emergencies_db.groupby(by=tst.emergencies_db['Type']).size()

Type
Bushfire    3
Flooding    2
dtype: int64

In [251]:
df = tst.emergencies_db.Type.value_counts().reset_index()
df.rename(columns = {'index':'Type', 'Type':'No of camps'}, inplace = True)
print("\nCamps in each area: \n",tabulate(df, headers='keys', tablefmt='psql', showindex=False))


Camps in each area: 
 +----------+---------------+
| Type     |   No of camps |
|----------+---------------|
| Bushfire |             3 |
| Flooding |             2 |
+----------+---------------+


In [255]:
emergency_options = tst.emergencies_db["Emergency ID"]
print('\n',*emergency_options, sep='\n')
emergency_list = tst.emergencies_db["Emergency ID"].tolist()



AU1
PK1
LBN1
PK2
AU2


In [256]:
print(emergency_options)

0     AU1
1     PK1
2    LBN1
3     PK2
4     AU2
Name: Emergency ID, dtype: object


In [260]:
df = tst.emergencies_db[["Emergency ID"]]
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+----------------+
| Emergency ID   |
|----------------|
| AU1            |
| PK1            |
| LBN1           |
| PK2            |
| AU2            |
+----------------+


In [265]:
camp_id = tst.refugee_db.loc[tst.refugee_db['Camp ID'].str.contains('AU1', case=False)]['Camp ID']
camp_id_list = camp_id.tolist()
print('\n',*set(camp_id), sep='\n')



AU1-1
AU1-2


In [269]:
df = tst.refugee_db.loc[tst.refugee_db['Camp ID'].str.contains('AU1', case=False)][['Camp ID']]
df.drop_duplicates(subset = 'Camp ID', inplace = True)
df

,Camp ID
0,AU1-1
1,AU1-2


In [277]:
print(tabulate(tst.refugee_db.tail(1), headers='keys', tablefmt='psql', showindex=False))

+-------------+---------------------------+------------------------------+-----------+----------------+------------------+-------------------------+
| Family ID   | Lead Family Member Name   | Lead Family Member Surname   | Camp ID   | Mental State   | Physical State   |   No. Of Family Members |
|-------------+---------------------------+------------------------------+-----------+----------------+------------------+-------------------------|
| 3AU1-2      | Sophie                    | Jones                        | AU1-2     | bad            | bad              |                       2 |
+-------------+---------------------------+------------------------------+-----------+----------------+------------------+-------------------------+


In [279]:
print('\nList of Camp IDs: ')
print(tst.camps_db['Camp ID'].to_string(index=False))


List of Camp IDs: 
AU1-1
AU1-2
PK1-1
PK1-2
AU1-3


In [282]:
print('\nList of Camp IDs: ')
print(tabulate(tst.camps_db[['Camp ID']], headers='keys', tablefmt='psql', showindex=False))


List of Camp IDs: 
+-----------+
| Camp ID   |
|-----------|
| AU1-1     |
| AU1-2     |
| PK1-1     |
| PK1-2     |
| AU1-3     |
+-----------+


In [293]:
vol_data = tst.vol_db
answers = ['Fedor','Sulitskiy']
df = vol_data[(vol_data['First name'] == answers[0].title()) & (vol_data['Second name'] == answers[1].title())]
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+------------+--------------+---------------+---------+-----------+----------------+
| Username   | First name   | Second name   | Phone   | Camp ID   | Availability   |
|------------+--------------+---------------+---------+-----------+----------------|
+------------+--------------+---------------+---------+-----------+----------------+


In [305]:
emergency_df = tst.emergencies_db
emergency_df.loc[emergency_df['Emergency ID'].str.contains('AU', case=False)]

,Emergency ID,Location,Type,Description,Start date,Close date
0,AU1,Australia,Bushfire,Description,01/09/2019,01/03/2020
4,AU2,Australia,Bushfire,Description,06/06/2022,


In [300]:
df = tst.emergencies_db.iloc[-1]
df

Emergency ID            AU2
Location          Australia
Type               Bushfire
Description     Description
Start date       06/06/2022
Close date                 
Name: 4, dtype: object

In [302]:
print(tabulate(tst.emergencies_db.tail(1), headers='keys', tablefmt='psql', showindex=False))

+----------------+------------+----------+---------------+--------------+--------------+
| Emergency ID   | Location   | Type     | Description   | Start date   | Close date   |
|----------------+------------+----------+---------------+--------------+--------------|
| AU2            | Australia  | Bushfire | Description   | 06/06/2022   |              |
+----------------+------------+----------+---------------+--------------+--------------+
